In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button, messagebox
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

class YOLOZoneDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YOLO Zone Detection")
        self.root.geometry("900x650")
        self.root.config(bg="#f1faee")  # Background color

        # Make grid expand to center all content
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=1)
        self.root.grid_rowconfigure(2, weight=1)
        self.root.grid_rowconfigure(3, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_columnconfigure(1, weight=1)

        # Initialize variables
        self.image_path = None
        self.model = None

        # Create the UI
        self.create_widgets()

    def create_widgets(self):
        """Set up the user interface components."""

        # Load and display the banner image
        banner_img = Image.open(r"C:\Users\mousa\Desktop\samam.png")  # Use a raw string
        banner_img.thumbnail((400, 150))  # Resize the image to fit
        banner_img_tk = ImageTk.PhotoImage(banner_img)

        # Banner label to display the image
        self.banner_label = Label(self.root, image=banner_img_tk, bg="#f1faee")
        self.banner_label.image = banner_img_tk  # Keep a reference to avoid garbage collection
        self.banner_label.grid(row=0, column=0, columnspan=2, pady=20, sticky="n")

        # Image display label (inside a frame for better control)
        self.image_frame = tk.Frame(self.root, bg="#34495E", bd=2, relief="groove")
        self.image_frame.grid(row=1, column=0, columnspan=2, padx=20, pady=10)

        self.image_label = Label(self.image_frame, bg="#34495E")
        self.image_label.pack(padx=10, pady=10)

        # Load model button
        load_model_button = Button(self.root, text="Load YOLO Model", command=self.load_model, font=("Helvetica", 12), bg="#1d3557", fg="white", padx=10, pady=5)
        load_model_button.grid(row=2, column=0, padx=10, pady=10, sticky="ew")

        # Upload image button
        upload_button = Button(self.root, text="Upload Image", command=self.upload_image, font=("Helvetica", 12), bg="#e63946", fg="white", padx=10, pady=5)
        upload_button.grid(row=2, column=1, padx=10, pady=10, sticky="ew")

        # Run detection button
        detect_button = Button(self.root, text="Run Detection", command=self.run_detection, font=("Helvetica", 12), bg="#457b9d", fg="white", padx=10, pady=5)
        detect_button.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

    def load_model(self):
        """Load YOLO model from file."""
        model_path = filedialog.askopenfilename(filetypes=[("YOLO Model", "*.pt")])

        if model_path:
            try:
                self.model = YOLO(model_path)
                messagebox.showinfo("Success", f"Model loaded: {model_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load model:\n{e}")
        else:
            messagebox.showinfo("Info", "Model loading canceled.")

    def upload_image(self):
        """Upload and display an image."""
        self.image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])

        if self.image_path:
            try:
                img = Image.open(self.image_path)
                img.thumbnail((800, 400))  # Resize for display
                img_tk = ImageTk.PhotoImage(img)

                # Update image label
                self.image_label.config(image=img_tk)
                self.image_label.image = img_tk
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image:\n{e}")
        else:
            messagebox.showinfo("Info", "Image upload canceled.")

    def run_detection(self):
        """Run YOLO detection on the uploaded image."""
        if not self.model:
            messagebox.showerror("Error", "Please load a YOLO model first.")
            return

        if not self.image_path:
            messagebox.showerror("Error", "Please upload an image first.")
            return

        # Load the image using OpenCV
        img = cv2.imread(self.image_path)

        try:
            # Perform detection
            results = self.model(img)
            annotated_img = results[0].plot()

            # Convert the annotated image to RGB for Tkinter display
            annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
            annotated_img_pil = Image.fromarray(annotated_img_rgb)
            annotated_img_pil.thumbnail((800, 400))

            # Update image label with detected image
            img_tk = ImageTk.PhotoImage(annotated_img_pil)
            self.image_label.config(image=img_tk)
            self.image_label.image = img_tk

        except Exception as e:
            messagebox.showerror("Error", f"Detection failed:\n{e}")

# Create and run the Tkinter app
if __name__ == "__main__":
    root = tk.Tk()
    app = YOLOZoneDetectionApp(root)
    root.mainloop()



0: 384x640 2 potholes, 48.0ms
Speed: 4.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
